# LSTM Analysis

In [1]:
import h5py
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

Using TensorFlow backend.
/home/lucas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lucas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lucas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lucas/.local/lib/python3.6/site-packages/tensorflow/python/fra

## Read dataset

In [2]:
dataset_file = "../data/users.h5"
users_df = pd.read_hdf(dataset_file)
users_df = pd.DataFrame(users_df.to_records())
users_df = users_df
users_df

,time_hour,0,1,2,3,10,11,12,13,20,...,972,973,980,982,983,990,991,992,993,99300
0,2012-08-18 16:00:00,103,32,35,36,1919,667,610,642,1694,...,20,47,32,11,21,609,208,194,207,161282
1,2012-08-18 17:00:00,78,31,20,27,1348,462,429,457,1226,...,15,37,16,7,9,377,136,126,115,108600
2,2012-08-18 18:00:00,36,17,9,10,914,304,286,324,929,...,10,28,14,5,9,212,88,70,54,77076
3,2012-08-18 19:00:00,32,14,7,11,653,210,208,235,799,...,8,22,11,4,7,160,65,44,51,60118
4,2012-08-18 20:00:00,23,7,5,11,551,183,174,194,834,...,15,21,8,2,6,153,58,42,53,54592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,2012-08-26 11:00:00,235,84,94,57,846,357,188,301,3308,...,55,90,38,11,27,939,260,315,364,244852
188,2012-08-26 12:00:00,219,81,77,61,928,381,207,340,3314,...,64,88,41,16,25,941,278,313,350,241728
189,2012-08-26 13:00:00,187,67,63,57,771,303,172,296,2966,...,50,79,36,17,19,876,262,299,315,220012
190,2012-08-26 14:00:00,175,57,62,56,592,200,139,253,2572,...,41,90,33,14,19,794,276,233,285,198886


### LSTM Config

In [3]:
learning_rate = 0.01
window = 12
epochs = 500
batch_size = 32
num_input = len(users_df.columns[1:])
timesteps = users_df.shape[0]
keep_rate_DROPOUT = 1;
optimizer = Adam(learning_rate=learning_rate)
loss_function = "mean_squared_error"
lstm_units = 50
bs = "10"


In [4]:
print("Learning rate:                   ", learning_rate)
print("epochs:                          ", epochs)
print("Keep Rate DROPOUT:               ", keep_rate_DROPOUT)

print("Number of input(Metrics):        ", num_input)
print("Time steps:                      ", timesteps)
print("Batch size:                      ", batch_size)


Learning rate:                    0.01
epochs:                           500
Keep Rate DROPOUT:                1
Number of input(Metrics):         400
Time steps:                       192
Batch size:                       32


## LSTM Data pre-processing

In [5]:
dataset = users_df[users_df.columns[1:]].to_numpy()
dataset

array([[   103,     32,     35, ...,    194,    207, 161282],
       [    78,     31,     20, ...,    126,    115, 108600],
       [    36,     17,      9, ...,     70,     54,  77076],
       ...,
       [   187,     67,     63, ...,    299,    315, 220012],
       [   175,     57,     62, ...,    233,    285, 198886],
       [   159,     53,     56, ...,    223,    239, 159072]])

In [6]:
train_data_len = math.ceil(len(dataset) * .8)
train_data_len

154

In [7]:
#Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

array([[0.30258303, 0.23684211, 0.33707865, ..., 0.43603133, 0.30418251,
        0.28341841],
       [0.2103321 , 0.22807018, 0.16853933, ..., 0.25848564, 0.12927757,
        0.1546292 ],
       [0.05535055, 0.10526316, 0.04494382, ..., 0.11227154, 0.01330798,
        0.07756395],
       ...,
       [0.61254613, 0.54385965, 0.65168539, ..., 0.71018277, 0.5095057 ,
        0.42699288],
       [0.56826568, 0.45614035, 0.64044944, ..., 0.53785901, 0.45247148,
        0.37534714],
       [0.50922509, 0.42105263, 0.57303371, ..., 0.51174935, 0.36501901,
        0.27801572]])

In [8]:
train_data = scaled_data[0:train_data_len, :]
train_data.shape

(154, 400)

In [9]:
test_data = scaled_data[train_data_len - window:, :]
test_data.shape

(50, 400)

In [10]:
XTrain = []
YTrain = []

for i in range(window, len(train_data)):
    XTrain.append(train_data[i-window:i])
    YTrain.append(train_data[i])

In [11]:
XTrain, YTrain = np.array(XTrain), np.array(YTrain)
print(XTrain.shape)
print(YTrain.shape)

(142, 12, 400)
(142, 400)


In [12]:
m = Sequential()
m.add(LSTM(units=lstm_units, return_sequences=True, input_shape=(XTrain.shape[1], num_input)))
m.add(Dropout(keep_rate_DROPOUT))
m.add(LSTM(units=lstm_units))
m.add(Dropout(keep_rate_DROPOUT))
m.add(Dense(units=num_input))
m.compile(optimizer=optimizer, loss=loss_function)

In [ ]:
history = m.fit(XTrain, YTrain, epochs=epochs, batch_size=batch_size, verbose=1)


Epoch 1/500
142/142 [==============================] - 0s 3ms/step - loss: 0.1821
Epoch 2/500
142/142 [==============================] - 0s 339us/step - loss: 0.0757
Epoch 3/500
142/142 [==============================] - 0s 331us/step - loss: 0.0571
Epoch 4/500
142/142 [==============================] - 0s 344us/step - loss: 0.0366
Epoch 5/500
142/142 [==============================] - 0s 317us/step - loss: 0.0253
Epoch 6/500
142/142 [==============================] - 0s 300us/step - loss: 0.0205
Epoch 7/500
142/142 [==============================] - 0s 322us/step - loss: 0.0173
Epoch 8/500
142/142 [==============================] - 0s 380us/step - loss: 0.0158
Epoch 9/500
142/142 [==============================] - 0s 437us/step - loss: 0.0156
Epoch 10/500
142/142 [==============================] - 0s 335us/step - loss: 0.0145
Epoch 11/500
142/142 [==============================] - 0s 300us/step - loss: 0.0142
Epoch 12/500
142/142 [==============================] - 0s 285us/step - los

142/142 [==============================] - 0s 302us/step - loss: 0.0043
Epoch 96/500
142/142 [==============================] - 0s 284us/step - loss: 0.0041
Epoch 97/500
142/142 [==============================] - 0s 290us/step - loss: 0.0041
Epoch 98/500
142/142 [==============================] - 0s 286us/step - loss: 0.0041
Epoch 99/500
142/142 [==============================] - 0s 284us/step - loss: 0.0041
Epoch 100/500
142/142 [==============================] - 0s 279us/step - loss: 0.0042
Epoch 101/500
142/142 [==============================] - 0s 278us/step - loss: 0.0041
Epoch 102/500
142/142 [==============================] - 0s 297us/step - loss: 0.0040
Epoch 103/500
142/142 [==============================] - 0s 280us/step - loss: 0.0040
Epoch 104/500
142/142 [==============================] - 0s 284us/step - loss: 0.0039
Epoch 105/500
142/142 [==============================] - 0s 281us/step - loss: 0.0039
Epoch 106/500
142/142 [==============================] - 0s 279us/step -

142/142 [==============================] - 0s 283us/step - loss: 0.0028
Epoch 191/500
142/142 [==============================] - 0s 294us/step - loss: 0.0028
Epoch 192/500
142/142 [==============================] - 0s 292us/step - loss: 0.0028
Epoch 193/500
142/142 [==============================] - 0s 292us/step - loss: 0.0028
Epoch 194/500
142/142 [==============================] - 0s 281us/step - loss: 0.0028
Epoch 195/500
142/142 [==============================] - 0s 288us/step - loss: 0.0028
Epoch 196/500
142/142 [==============================] - 0s 283us/step - loss: 0.0028
Epoch 197/500
142/142 [==============================] - 0s 290us/step - loss: 0.0028
Epoch 198/500
142/142 [==============================] - 0s 289us/step - loss: 0.0028
Epoch 199/500
142/142 [==============================] - 0s 286us/step - loss: 0.0028
Epoch 200/500
142/142 [==============================] - 0s 286us/step - loss: 0.0028
Epoch 201/500
142/142 [==============================] - 0s 283us/st

In [ ]:
plt.figure()
plt.ylabel('loss');plt.xlabel=('epoch')
plt.semilogy(history.history['loss'])

In [ ]:
XTest = []

YTest = dataset[train_data_len:]
test_data
for i in range(window, len(test_data)):
    XTest.append(test_data[i-window:i])

XTest, YTest = np.array(XTest), np.array(YTest)

x_pred = m.predict(XTest)
x_pred = scaler.inverse_transform(x_pred)
prediction = pd.DataFrame(data=x_pred, columns=users_df.columns[1:])

In [ ]:
train = users_df[:train_data_len]
valid = users_df[train_data_len:]

prediction['time_hour'] = valid['time_hour'].values

prediction

In [ ]:
plt.figure(figsize=(20, 15))
users_df.plot(x='time_hour', y=bs)

In [ ]:
plt.figure(figsize=(20, 15))
plt.title('Model')
ax = plt.gca()
train.plot(x='time_hour', y=bs, ax=ax)
valid.plot(x='time_hour', y=bs, ax=ax)
prediction.plot(x='time_hour', y=bs, ax=ax)
plt.legend()

In [ ]:
plt.figure(figsize=(20, 15))
plt.title('Model')
ax = plt.gca()
valid.plot(x='time_hour', y=bs, ax=ax)
prediction.plot(x='time_hour', y=bs, ax=ax)
plt.legend()

In [ ]:
rmse = np.sqrt(np.mean(x_pred - YTest)**2)
rmse

In [ ]:
rmse = np.sqrt(np.mean(prediction[bs].values - valid[bs].values)**2)
rmse